In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler,FunctionTransformer,OrdinalEncoder, LabelEncoder, OneHotEncoder, MinMaxScaler, PowerTransformer, KBinsDiscretizer, Binarizer, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.impute import KNNImputer,SimpleImputer,MissingIndicator
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, r2_score, confusion_matrix, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import *
from sklearn.tree import plot_tree
from sklearn.cluster import AgglomerativeClustering

from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier,AdaBoostClassifier, StackingClassifier

from dtreeviz.trees import *

from imblearn.pipeline import make_pipeline

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

import scipy.fft as fft,scipy.fftpack as fftpack
import scipy.cluster.hierarchy as hc
import scipy.stats as stats
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from matplotlib.patches import FancyArrowPatch
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

import plotly.express as px
from IPython.display import Image, display_svg,SVG

import datetime
import statistics
import time

import optuna

import random

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

import requests

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import cv2

import os

import threading
import queue
import shutil
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [99]:
class Google_images_Web_scrapping:
    
    def __init__(self,chrome_driver_executablepath,folder_name,URL,Allimages_containerClass,previewImg_class,largeImg_xpath,proxy_file_loc):
        self.URL=URL
        self.folder_name=folder_name
        self.previewImg_class=previewImg_class
        self.largeImg_xpath=largeImg_xpath
        self.proxy_file_loc=proxy_file_loc
        self.Allimages_containerClass=Allimages_containerClass
        self.chrome_driver_executablepath=chrome_driver_executablepath
        
        # self.container_xpath=container_xpath
    def create_folder(self):
        if not os.path.isdir(self.folder_name):
            os.makedirs(self.folder_name) 
    
    def search_URL(self):
        global driver
        cService=webdriver.ChromeService(executable_path=self.chrome_driver_executablepath)
        driver=webdriver.Chrome(service=cService)
        driver.get(self.URL)
        time.sleep(1)
        
    def load_images(self,scroll_y):
        global driver
        driver.execute_script('window.scrollTo(0,%s);'%scroll_y)
        time.sleep(6)
        page_html=driver.page_source
        pageSoup=BeautifulSoup(page_html,'html.parser')
        containers=pageSoup.findAll("div",{"class":self.Allimages_containerClass})
        len_cont=len(containers)
        print(len_cont)
        return len_cont
    
    def download_img(self,url,num):
        global proxies
        response=requests.get(url,proxies={'http:':proxies,'https:':proxies})
        if response.status_code==200:
            with open(os.path.join(self.folder_name,str(num)+".jpg"),'wb') as file:
                file.write(response.content)
    
    def load_and_downloadimages(self,len_container):
        global count
        global images_count
        for i in range(count,len_container+1):
            xpath="""//*[@id="rso"]/div/div/div[1]/div/div/div[%s]"""%i
            previewImgPath="""//*[@id="rso"]/div/div/div[1]/div/div/div[%s]/a[1]/div[1]/g-img"""%i
            
            try:
                time.sleep(1)
                previewImgElement=driver.find_element(By.XPATH,previewImgPath)
            except:
                preview_ImgElement=driver.find_element(By.CLASS_NAME,self.previewImg_class)
                previewImgURL=preview_ImgElement.get_attribute('src')   
                driver.find_element(By.XPATH,xpath).click()
                time.sleep(3)
                timeStarted=time.time()
                cond=True
                while cond==True:
                    largeView_ImgElement=driver.find_element(By.XPATH,self.largeImg_xpath)
                    largeView_ImgURL=largeView_ImgElement.get_attribute('src') 
                    print(largeView_ImgURL)
                    if largeView_ImgURL!=previewImgURL:
                        cond=False
                    else:
                        currentTime=time.time()
                        if currentTime-timeStarted>10:
                            print("Time to load large image is greater than 10 seconds")
                            cond=False
                
                try:
                    self.download_img(largeView_ImgURL,i)
                    images_count+=1
                    print("Downloaded element %s out of %s total. URL: %s"%(i,len_container+1,largeView_ImgURL))
                    print(f'images_count={images_count}, count={count}')
                except:
                    print("Couldn't download image %s"%i)
        
    
    def get_images(self,number_of_images):
        global driver
        global images_count
        global count
        global proxies
        with open(self.proxy_file_loc,'r') as f:
            proxies=f.read().split('\n')
        images_count=0
        count=1
        scroll_y=1
        self.create_folder()
        self.search_URL()
        print('scroll_y=',scroll_y)
        len_container=self.load_images(str(scroll_y))
        self.load_and_downloadimages(len_container)
        count=len_container
        
        
        while images_count<number_of_images:
            scroll_y=scroll_y*10000
            print('scroll_y=',scroll_y)
            len_container=self.load_images(scroll_y)
            self.load_and_downloadimages(len_container)
            count=len_container

In [96]:
GoogleImgWebScrapping=Google_images_Web_scrapping(chrome_driver_executablepath='/Users/drago/Documents/chromedriver-mac-arm64/chromedriver',
                                                  folder_name='Messi images',
                                                  URL='https://www.google.com/search?q=messi&sca_esv=d609cd469e173d32&udm=2&biw=1280&bih=659&sxsrf=ADLYWILVFjfj47il1b7q3vzf4kH0bQ5fPg%3A1736381548241&ei=bBR_Z8mzDq-q4-EP2ujQmA8&ved=0ahUKEwjJmYKrreeKAxUv1TgGHVo0FPMQ4dUDCBA&uact=5&oq=messi&gs_lp=EgNpbWciBW1lc3NpMgQQIxgnMgQQIxgnMgoQABiABBhDGIoFMgoQABiABBhDGIoFMgoQABiABBhDGIoFMgoQABiABBhDGIoFMgoQABiABBhDGIoFMgoQABiABBhDGIoFMgoQABiABBhDGIoFMg0QABiABBixAxhDGIoFSLIJUABYtwVwAHgAkAEAmAF8oAHXBKoBAzAuNbgBA8gBAPgBAZgCBaAC7gTCAggQABiABBixA8ICCxAAGIAEGLEDGIMBwgIFEAAYgATCAg4QABiABBixAxiDARiKBcICChAAGIAEGLEDGAqYAwCSBwMwLjWgB9ck&sclient=img',
                                                  Allimages_containerClass="eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd",
                                                  previewImg_class="YQ4gaf",
                                                  largeImg_xpath='//*[@id="Sva75c"]/div[2]/div[2]/div/div[2]/c-wiz/div/div[3]/div[1]/a/img[1]',
                                                  proxy_file_loc='/Users/drago/Documents/Practice files/Data_files/valid_proxies.txt')

In [ ]:
GoogleImgWebScrapping.get_images(500)

In [75]:
trial=trial()
